# The EDSII Construction Notebook

This notebook provides code used for the construction of the European Digital Social Innovation Index. 

The data can be downloaded from: https://www.nesta.org.uk/documents/1395/Downloadable_data_EDSII.xlsx. 

For more information about the EDSII, including a detailed methodology, discussion of key results, examples of how governments are already supporting DSI and an interactive data visualisation, see: https://www.nesta.org.uk/feature/european-digital-social-innovation-index/.

# Import packages and data

In [1]:
# Import packages
import pandas as pd
import numpy as np
from fancyimpute import IterativeImputer
from sklearn.preprocessing import StandardScaler
import requests
from io import BytesIO

# Import data
URL = 'https://media.nesta.org.uk/documents/Downloadable_data_EDSII.xlsx'
r = requests.get(URL)

with BytesIO(r.content) as f:
    standardised_data = pd.read_excel(f, sheet_name = 'Data', encoding = "ISO-8859-1")[1:61]
    colnames = pd.read_excel(f, sheet_name = 'Data').iloc[0]

colnames[0:2] = ['City', 'Country']
standardised_data.columns = colnames

C:\Users\jbone\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'<?xml ve'

# Define functions

In [ ]:
# This function removes outliers by transforming high (low) outliers to have the same value as the highest (lowest) existing value in that variable which falls above Q3 + 1.5 IQR or below Q1 − 1.5 IQR. 
def windsor(df):
    windsored_data = df.copy()
    for column in windsored_data.iloc[:, 2:]:
        Q1 = windsored_data[column].quantile(0.25)
        Q3 = windsored_data[column].quantile(0.75)
        Max = max(windsored_data[column].dropna())
        Min = min(windsored_data[column].dropna())
        IQR = Q3 - Q1
        upperlimit = Q3 + 1.5 * IQR
        lowerlimit = Q1 - 1.5 * IQR
        Max_nonoutlier = max(windsored_data[windsored_data[column] < upperlimit][column].dropna())
        Min_nonoutlier = min(windsored_data[windsored_data[column] > lowerlimit][column].dropna())
        while Max > upperlimit:
            windsored_data[column] = windsored_data[column].replace(Max,Max_nonoutlier)
            Max = max(windsored_data[column].dropna())
        while Min < lowerlimit:
            windsored_data[column] = windsored_data[column].replace(Min,Min_nonoutlier)
            Min = max(windsored_data[column].dropna())
    return windsored_data
             
# This function normalises columns to be within an identical min-max range of [0.1, 0.9].
def normalise(df):
    normalized_data = df.copy()
    for column in df.columns:
        max_value = df[column].max()
        min_value = df[column].min()
        normalized_data[column] = (0.8 * (df[column] - min_value) / (max_value - min_value))+0.1
    return normalized_data

# This function aggregates indicators into themes and themes into a final index score
def aggregate(df_in, varweights, bucketweights):
    df_copy = df_in.copy()
    colnames = df_copy.columns
    # Apply variable weights & artihmetic mean
    for i in range(0,len(colnames)):
        df_copy[colnames[i]] = df_in[df_in.columns[i]]*varweights[i]
    combs = [[0,1,2,3,4,5],[6,7,8,9,10,11],[12,13,14,15,16],[17,18,19,20,21,22],[23,24,25,26,27],[28,29,30,31]]
    count=0
    for l in combs:
        cols = l
        count +=1
        newcol = "B" + str(count)
        df_copy[newcol] = df_copy.iloc[:,cols].sum(axis=1)/sum(varweights[cols[0]:(cols[-1]+1)])
    theme_scores = df_copy.iloc[:,-6:]
    theme_scores.columns = ['Civil Society', 'Collaboration', 'Skills', 'Infrastructure', 'Funding', 'Diversity and Inclusion']
    df_copy = normalise(df_copy)
    # Apply bucket weights & geometric mean
    for j in range(0,6):
        oldcol = "B" + str(j + 1)
        newcol = "C" + str(j + 1)
        df_copy[newcol] = df_copy[oldcol] ** bucketweights[j]
    collist = [x for x in range(38,len(df_copy.columns))]
    list_out = list(df_copy.iloc[:,collist].prod(axis=1)**(1/sum(bucketweights)))
    return [list_out,theme_scores]

## Data cleaning

In [ ]:
# Transformation of outliers
windsored_data = windsor(standardised_data) 

# Normalisaion of columns to identical min-max range of [0.1, 0.9]
normalized_data = normalise(windsored_data.iloc[:, 2:])
normalized_data = pd.DataFrame(normalized_data, dtype='float')

# Imputation of missing values
# Some indicators are made up of several measures. Where this is the case we merge measures into a single indicator by taking their arithmetic mean.
normalized_data['Government collaboration with tech sector'] = normalized_data[['Government collaboration with tech sector 1','Government collaboration with tech sector 2','Government collaboration with tech sector 3']].mean(axis=1)
normalized_data['Diversity within the tech sector'] = normalized_data[['Diversity within the tech sector 1','Diversity within the tech sector 2']].mean(axis=1)
normalized_data['Diversity within civil society'] = normalized_data[['Diversity within civic society 1','Diversity within civic society 2']].mean(axis=1)
normalized_data['Openness of data'] = normalized_data[['Openness of data 1','Openness of data 2']].mean(axis=1)
normalized_data['Access to Business, HR, legal, marketing, design and media support'] = normalized_data[['Access to Business, HR, legal, marketing, design and media support 1','Access to Business, HR, legal, marketing, design and media support 2','Access to Business, HR, legal, marketing, design and media support 3','Access to Business, HR, legal, marketing, design and media support 4','Access to Business, HR, legal, marketing, design and media support 5']].mean(axis=1)
normalized_data['Social cohesion'] = normalized_data[['Social cohesion 1','Social cohesion 2','Social cohesion 3','Social cohesion 4']].mean(axis=1)
normalized_data['Access to fast broadband and mobile internet 1'] = normalized_data[['Access to fast broadband and mobile internet 1','Access to fast broadband and mobile internet 2','Access to fast broadband and mobile internet 3']].mean(axis=1)
normalized_data['Access to fast broadband and mobile internet 2'] = normalized_data[['Access to fast broadband and mobile internet 4','Access to fast broadband and mobile internet 5','Access to fast broadband and mobile internet 6']].mean(axis=1)
normalized_data['Access to fast broadband and mobile internet'] = normalized_data[['Access to fast broadband and mobile internet 1','Access to fast broadband and mobile internet 2']].mean(axis=1)
normalized_data = normalized_data[['Access to Volunteers', 'Positve attitudes to civil society', 'Social cohesion','Individual giving', 'Public advocacy for DSI','Presence of supportive governent policy for social purpose inititives', 'Events where people can meet to network / discuss DSI', 'Online collaboration','Tech sector collaboration with civil society','Government collaboration with civic society', 'Government collaboration with tech sector','Engagement with DSI','Access to Business, HR, legal, marketing, design and media support','Access to employees with data skills','Access to employees with service design skills','Access to employees with Software Engineering / Development skills','Presence of research institutions with expertise in DSI (standardized by pop.)','Access to fast broadband and mobile internet','Access to flexible work space','Openness of data','Access to fabrication and manufactoring facilities','Presence of socially focussed business support','Ease of starting a business', 'Availability of Seed Grant Funding','Availability of Major Grant Funding', 'Flexibility of funding','Availability of impact investment','Willingness of public and social sector procure from SMEs','Diversity within the tech sector', 'Diversity within civil society','Inclusivity of innovation', 'Digital inclusion']]

#Replace missing values replaced with estimated values
XY_incomplete = normalized_data

n_imputations = 5
XY_completed = []
for i in range(n_imputations):
    imputer = IterativeImputer(n_iter=5, sample_posterior=True, random_state=i)
    XY_completed.append(imputer.fit_transform(XY_incomplete))

XY_completed_mean = np.mean(XY_completed, 0)
XY_completed_std = np.std(XY_completed, 0)
imputed_data = pd.DataFrame(XY_completed_mean)
imputed_data.columns = normalized_data.columns
imputed_data.index = normalized_data.index

## Weighting and aggregation

In [ ]:
# Indicator weights
varweights_stup = [18.18, 18.18, 18.18, 9.10, 18.18, 18.18, 5.26, 10.52, 21.05, 21.05, 21.05, 21.05, 20, 20, 20, 20, 20, 22.22, 16.67, 22.22, 11.11, 16.67, 11.11, 16.67, 8.33, 25, 25, 25, 22.22, 22.22, 22.22, 33.34]
bucketweights_stup = [20, 17.5, 17.5, 15, 15, 15]
df_normin = imputed_data

# Calculation of index scores
results_stup = aggregate(df_normin, varweights_stup, bucketweights_stup)
index_score = pd.DataFrame(results_stup[0], columns=['Index Score'], index=df_normin.index)
for column in index_score.columns:
         index_score[column] = ((index_score[column] - 0.1) / (0.8))# rescaling to same scale as presented online       
index_score = pd.concat([standardised_data.iloc[:, :1],index_score],axis=1)
theme_scores = pd.DataFrame(results_stup[1], columns=['Civil Society', 'Collaboration', 'Skills', 'Infrastructure', 'Funding', 'Diversity and Inclusion'], index=df_normin.index)
for column in theme_scores.columns:
         theme_scores[column] = ((theme_scores[column] - 0.1) / (0.8))# rescaling to same scale as presented online     
theme_scores = pd.concat([standardised_data.iloc[:, :1],theme_scores],axis=1)

In [2]:
#print final index score
print(index_score)

#write index score to .csv
index_score.to_csv('index_score.csv')

NameError: name 'index_score' is not defined

In [ ]:
#print final theme scores
print(theme_scores)

#write theme score to .csv
theme_scores.to_csv('theme_scores.csv')